## Playground 🤟🏼

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2024-07-08 08:11:08.066221: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 08:11:08.066322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 08:11:08.197819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Loading the Dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist
for sets in [X_train, y_train, X_test, y_test]:
    print(sets.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# Creating the Validation Set
X_train, X_valid = X_train[:-8000], X_train[-8000:]
y_train, y_valid = y_train[:-8000], y_train[-8000:]

for sets in [X_train, y_train, X_valid, y_valid, X_test, y_test]:
    print(sets.shape)

(52000, 28, 28)
(52000,)
(8000, 28, 28)
(8000,)
(10000, 28, 28)
(10000,)


In [4]:
class SparseCategoricalCrossentropy(tf.keras.losses.Loss):
    def __init__(self, from_logits=False, reduction="sum_over_batch_size",
                name="sparse_categoricial_crossentropy"):
        super().__init__(reduction=reduction, name=name)
        self.from_logits = from_logits
        
    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.int64)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=self.from_logits)
        return loss
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "from_logits": self.from_logits}

In [5]:
tf.keras.backend.clear_session()

simple_loss_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), input_shape=[28, 28, 1], activation="elu"),
    tf.keras.layers.Conv2D(32, (3, 3), activation="elu"),
    tf.keras.layers.MaxPool2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation="elu"),
    tf.keras.layers.Conv2D(64, (3, 3), activation="elu"),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(512, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(256, activation="elu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(10, activation="softmax")
])

simple_loss_model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,559,210 (9.76 MB)

 Trainable params: 2,559,210 (9.76 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
exponential_sch = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10e2,
    decay_rate=0.1,
    staircase=False
)

# Using the custom loss function for compilation
simple_loss_model.compile(
    loss=SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
    optimizer=tf.keras.optimizers.Nadam(learning_rate=exponential_sch)
)

In [7]:
# Normalising and Standardising the Images
X_train_mean = X_train.mean()
X_train_std = X_train.std()

for sets in [X_train, X_valid, X_test]:
    sets = (sets - X_train_mean) / X_train_std
    sets = sets / 255

In [8]:
# Early Stopping
earlystop_cb = tf.keras.callbacks.EarlyStopping(
    patience=7, restore_best_weights=True, mode="min", monitor="val_loss", min_delta=1e-3
)

# Training using the Custom Loss
simple_loss_model.fit(
    X_train, y_train, validation_data=(X_valid, y_valid), epochs=30, callbacks=[earlystop_cb]
)

Epoch 1/30


2024-07-08 08:11:25.679136: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 2.35363, expected 1.53764
2024-07-08 08:11:25.679181: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 1.82107, expected 1.00508
2024-07-08 08:11:25.679194: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 1.81936, expected 1.00338
2024-07-08 08:11:25.679208: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.23827, expected 1.42229
2024-07-08 08:11:25.679219: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 3.05481, expected 2.23883
2024-07-08 08:11:25.679230: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 2.58554, expected 1.76955
2024-07-08 08:11:25.679240: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 3.00002, expected 2.18403
2024-07-08 08:11:25.679251: E external/local_xla/xla/se

  51/1625 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.1766 - loss: 313.0068

I0000 00:00:1720426288.325243      68 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720426288.347696      68 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1616/1625 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8357 - loss: 26.0968

W0000 00:00:1720426293.900050      66 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1625/1625 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8363 - loss: 25.9657 - val_accuracy: 0.9787 - val_loss: 0.0672
Epoch 2/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9827 - loss: 0.0560 - val_accuracy: 0.9812 - val_loss: 0.0633
Epoch 3/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9836 - loss: 0.0514 - val_accuracy: 0.9816 - val_loss: 0.0631
Epoch 4/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9835 - loss: 0.0524 - val_accuracy: 0.9818 - val_loss: 0.0631
Epoch 5/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9843 - loss: 0.0513 - val_accuracy: 0.9818 - val_loss: 0.0631
Epoch 6/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9843 - loss: 0.0517 - val_accuracy: 0.9818 - val_loss: 0.0631
Epoch 7/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9841 - loss: 0.0509 - val_accuracy: 0.9818 - val_loss: 0.0631
Epoch 8/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9839 - loss: 0.0505 - val_accur

In [9]:
simple_loss_model.save("my_simple_model.keras")

reloaded_model = tf.keras.models.load_model(
    "my_simple_model.keras",
    custom_objects={
        "SparseCategoricalCrossentropy": SparseCategoricalCrossentropy()
    }
)

reloaded_model.fit(
    X_train, y_train, validation_data=(X_valid, y_valid), epochs=10
)

Epoch 1/10
  51/1625 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9888 - loss: 0.0392

W0000 00:00:1720426340.542886      68 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1622/1625 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9846 - loss: 0.0500

W0000 00:00:1720426345.887915      67 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1625/1625 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9846 - loss: 0.0501 - val_accuracy: 0.9812 - val_loss: 0.0633
Epoch 2/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9839 - loss: 0.0506 - val_accuracy: 0.9812 - val_loss: 0.0633
Epoch 3/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9835 - loss: 0.0529 - val_accuracy: 0.9812 - val_loss: 0.0633
Epoch 4/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9840 - loss: 0.0523 - val_accuracy: 0.9812 - val_loss: 0.0633
Epoch 5/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9841 - loss: 0.0523 - val_accuracy: 0.9812 - val_loss: 0.0633
Epoch 6/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9842 - loss: 0.0501 - val_accuracy: 0.9812 - val_loss: 0.0633
Epoch 7/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9846 - loss: 0.0501 - val_accuracy: 0.9812 - val_loss: 0.0633
Epoch 8/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9841 - loss: 0.0504 - val_accurac

In [10]:
# Applying Streamlining Metrics
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

print(precision([1, 0, 1, 1], [1, 0, 1, 0]))
print(recall([1, 0, 1, 1], [1, 0, 1, 0]))

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(0.6666667, shape=(), dtype=float32)


In [11]:
# Building a Custom Layer
class MyDense(tf.keras.layers.Layer):
    def __init__(self, name, units, activation="relu", **kwargs):
        super().__init__(name=name, **kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", initializer="he_normal", shape=[batch_input_shape[-1], self.units]
        )
        
        self.bias = self.add_weight(
            name="bias", initializer="zeros", shape=[self.units]
        )
        
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, "activation": tf.keras.activations.serialize(self.activation)}

In [12]:
new_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="elu", input_shape=[28, 28, 1]),
    tf.keras.layers.Conv2D(32, (3, 3), activation="elu"),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    MyDense("Layer1", 512, "elu"),
    MyDense("Layer2", 512, "elu"),
    MyDense("Layer3", 512, "elu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

new_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Layer1 (MyDense)                │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Layer2 (MyDense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Layer3 (MyDense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,899,818 (11.06 MB)

 Trainable params: 2,899,818 (11.06 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
new_model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Nadam(learning_rate=exponential_sch),
    metrics=["accuracy"]
)

new_model.fit(
    X_train, y_train, validation_data=(X_valid, y_valid), epochs=30
)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:602: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1625/1625 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8755 - loss: 9.6732 - val_accuracy: 0.9818 - val_loss: 0.0710
Epoch 2/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9862 - loss: 0.0442 - val_accuracy: 0.9821 - val_loss: 0.0659
Epoch 3/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9897 - loss: 0.0368 - val_accuracy: 0.9822 - val_loss: 0.0657
Epoch 4/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9896 - loss: 0.0355 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 5/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9896 - loss: 0.0349 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 6/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9895 - loss: 0.0355 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 7/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9896 - loss: 0.0358 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 8/30
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9897 - loss: 0.0356 - val_accurac

In [14]:
new_model.save("new_saved_model.keras")

new_reload = tf.keras.models.load_model(
    "new_saved_model.keras",
    custom_objects={
        "MyDense": MyDense,
        "SparseCategoricalCrossentropy": SparseCategoricalCrossentropy
    }
)

new_reload.fit(
    X_train, y_train, validation_data=(X_valid, y_valid), epochs=10
)

Epoch 1/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9897 - loss: 0.0356 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 2/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9897 - loss: 0.0346 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 3/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9898 - loss: 0.0353 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 4/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9891 - loss: 0.0372 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 5/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9906 - loss: 0.0345 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 6/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9889 - loss: 0.0358 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 7/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9902 - loss: 0.0356 - val_accuracy: 0.9824 - val_loss: 0.0657
Epoch 8/10
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9887 - loss: 0.0368 - 